# Multiple Linear Regression in Statsmodels - Lab

## Introduction
In this lab, you'll practice fitting a multiple linear regression model on our Boston Housing Data set!

## Objectives
You will be able to:
* Run linear regression on Boston Housing dataset with all the predictors
* Interpret the parameters of the multiple linear regression model

## The Boston Housing Data

We pre-processed the Boston Housing Data again. This time, however, we did things slightly different:
- We dropped "ZN" and "NOX" completely
- We categorized "RAD" in 3 bins and "TAX" in 4 bins
- We used min-max-scaling on "B", "CRIM" and "DIS" (and logtransformed all of them first, except "B")
- We used standardization on "AGE", "INDUS", "LSTAT" and "PTRATIO" (and logtransformed all of them first, except for "AGE") 

In [57]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
boston_features = boston_features.drop(["NOX","ZN"],axis=1)

# first, create bins for based on the values observed. 3 values will result in 2 bins
bins = [0,6,  24]
bins_rad = pd.cut(boston_features['RAD'], bins)
bins_rad = bins_rad.cat.as_unordered()

# first, create bins for based on the values observed. 4 values will result in 3 bins
bins = [0, 270, 360, 712]
bins_tax = pd.cut(boston_features['TAX'], bins)
bins_tax = bins_tax.cat.as_unordered()

tax_dummy = pd.get_dummies(bins_tax, prefix="TAX")
rad_dummy = pd.get_dummies(bins_rad, prefix="RAD")
boston_features = boston_features.drop(["RAD","TAX"], axis=1)
boston_features = pd.concat([boston_features, rad_dummy, tax_dummy], axis=1)

In [58]:
age = boston_features["AGE"]
b = boston_features["B"]
logcrim = np.log(boston_features["CRIM"])
logdis = np.log(boston_features["DIS"])
logindus = np.log(boston_features["INDUS"])
loglstat = np.log(boston_features["LSTAT"])
logptratio = np.log(boston_features["PTRATIO"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["CRIM"] = (logcrim-min(logcrim))/(max(logcrim)-min(logcrim))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["AGE"] = (age-np.mean(age))/np.sqrt(np.var(age))
boston_features["INDUS"] = (logindus-np.mean(logindus))/np.sqrt(np.var(logindus))
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))
boston_features["PTRATIO"] = (logptratio-np.mean(logptratio))/(np.sqrt(np.var(logptratio)))

In [59]:
boston_features['RAD0to6'] = boston_features['RAD_(0, 6]'] 
boston_features['RAD6to24'] = boston_features['RAD_(6, 24]']
boston_features['TAX0to270'] = boston_features['TAX_(0, 270]'] 
boston_features['TAX270to360'] = boston_features['TAX_(270, 360]'] 
boston_features['TAX360to712'] = boston_features['TAX_(360, 712]']

boston_features= boston_features.drop(columns=['RAD_(0, 6]', 'RAD_(6, 24]', 'TAX_(0, 270]', 'TAX_(270, 360]', 'TAX_(360, 712]'])

In [40]:
boston_features.head()

,CRIM,INDUS,CHAS,RM,AGE,DIS,PTRATIO,B,LSTAT,RAD0to6,RAD6to24,TAX0to270,TAX270to360,TAX360to712
0,0.000000,-1.704344,0.0,6.575,-0.120013,0.542096,-1.443977,1.000000,-1.275260,1,0,0,1,0
1,0.153211,-0.263239,0.0,6.421,0.367166,0.623954,-0.230278,1.000000,-0.263711,1,0,1,0,0
2,0.153134,-0.263239,0.0,7.185,-0.265812,0.623954,-0.230278,0.989737,-1.627858,1,0,1,0,0
3,0.171005,-1.778965,0.0,6.998,-0.809889,0.707895,0.165279,0.994276,-2.153192,1,0,1,0,0
4,0.250315,-1.778965,0.0,7.147,-0.511180,0.707895,0.165279,1.000000,-1.162114,1,0,1,0,0


## Run an linear model in Statsmodels

In [41]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [42]:
boston_target= pd.DataFrame(boston.target)
boston_target['target']=boston_target[0]
boston_target = boston_target.drop(columns=0)
boston_full = pd.concat([boston_features, boston_target], axis=1)

In [45]:
features = "+".join(boston_features.columns)
formula = 'target~'+features

model = ols(formula=formula, data=boston_full).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.779
Model:                            OLS   Adj. R-squared:                  0.774
Method:                 Least Squares   F-statistic:                     144.9
Date:                Mon, 10 Dec 2018   Prob (F-statistic):          5.15e-153
Time:                        20:37:16   Log-Likelihood:                -1458.2
No. Observations:                 506   AIC:                             2942.
Df Residuals:                     493   BIC:                             2997.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       4.4397      1.785      2.487      0.013       0.933       7.947
CRIM           -1.9000      2.091     -0.909      0.364      -6.009       2.209
INDUS          -0.8069      0.362     -2.228      0.026      -1.518      -0.095
CHAS            2.5968      0.796      3.262      0.001       1.033       4.161
RM              2.6445      0.408      6.480      0.000       1.843       3.446
AGE             0.0787      0.352      0.224      0.823      -0.612       0.770
DIS           -10.0839      1.855     -5.437      0.000     -13.728      -6.440
PTRATIO        -1.4864      0.241     -6.159      0.000      -1.961      -1.012
B               3.8623      0.981      3.935      0.000       1.934       5.791
LSTAT          -5.6315      0.354    -15.929      0.000      -6.326      -4.937
RAD0to6         1.5563      0.821      1.896      0.059      -0.056       3.169
RAD6to24        2.8834      1.069      2.697      0.007       0.783       4.984
TAX0to270       2.6166      0.715      3.661      0.000       1.212       4.021
TAX270to360     1.3553      0.702      1.930      0.054      -0.025       2.735
TAX360to712     0.4679      0.683      0.685      0.493      -0.873       1.809
==============================================================================
Omnibus:                      106.736   Durbin-Watson:                   1.093
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              431.931
Skew:                           0.891   Prob(JB):                     1.61e-94
Kurtosis:                       7.161   Cond. No.                     6.27e+16
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 5.55e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

## Run the same model in Scikit-learn

In [46]:
from sklearn.linear_model import LinearRegression
y = boston_full['target']
linereg = LinearRegression()
linereg.fit(boston_features, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [47]:
linereg.coef_

array([ -1.89995316,  -0.80688617,   2.59684028,   2.64453176,
         0.0786663 , -10.0839112 ,  -1.48638161,   3.86233002,
        -5.63145746,  -0.66357383,   0.66357383,   1.13664819,
        -0.12462051,  -1.01202768])

In [48]:
linereg.intercept_

8.13952370766285

## Remove the necessary variables to make sure the coefficients are the same for Scikit-learn vs Statsmodels

In [49]:
boston_full = boston_full.drop(columns=['TAX360to712', 'RAD6to24'])


In [50]:
boston_features = boston_features.drop(columns=['TAX360to712', 'RAD6to24'])

### Statsmodels

In [51]:
features = "+".join(boston_features.columns)
formula = 'target~'+features

model = ols(formula=formula, data=boston_full).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.779
Model:                            OLS   Adj. R-squared:                  0.774
Method:                 Least Squares   F-statistic:                     144.9
Date:                Mon, 10 Dec 2018   Prob (F-statistic):          5.15e-153
Time:                        20:42:31   Log-Likelihood:                -1458.2
No. Observations:                 506   AIC:                             2942.
Df Residuals:                     493   BIC:                             2997.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       7.7911      3.406      2.288      0.023       1.100      14.482
CRIM           -1.9000      2.091     -0.909      0.364      -6.009       2.209
INDUS          -0.8069      0.362     -2.228      0.026      -1.518      -0.095
CHAS            2.5968      0.796      3.262      0.001       1.033       4.161
RM              2.6445      0.408      6.480      0.000       1.843       3.446
AGE             0.0787      0.352      0.224      0.823      -0.612       0.770
DIS           -10.0839      1.855     -5.437      0.000     -13.728      -6.440
PTRATIO        -1.4864      0.241     -6.159      0.000      -1.961      -1.012
B               3.8623      0.981      3.935      0.000       1.934       5.791
LSTAT          -5.6315      0.354    -15.929      0.000      -6.326      -4.937
RAD0to6        -1.3271      0.669     -1.983      0.048      -2.642      -0.012
TAX0to270       2.1487      0.704      3.051      0.002       0.765       3.532
TAX270to360     0.8874      0.607      1.461      0.145      -0.306       2.081
==============================================================================
Omnibus:                      106.736   Durbin-Watson:                   1.093
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              431.931
Skew:                           0.891   Prob(JB):                     1.61e-94
Kurtosis:                       7.161   Cond. No.                         126.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Scikit-learn

In [52]:
y = boston_full['target']
linereg = LinearRegression()
linereg.fit(boston_features, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [53]:
linereg.coef_

array([ -1.89995316,  -0.80688617,   2.59684028,   2.64453176,
         0.0786663 , -10.0839112 ,  -1.48638161,   3.86233002,
        -5.63145746,  -1.32714767,   2.14867587,   0.88740717])

In [54]:
linereg.intercept_

7.791069861812664

## Interpret the coefficients for PTRATIO, PTRATIO, LSTAT

- CRIM: per capita crime rate by town
- INDUS: proportion of non-retail business acres per town
- CHAS: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
- RM: average number of rooms per dwelling
- AGE: proportion of owner-occupied units built prior to 1940
- DIS: weighted distances to five Boston employment centres
- RAD: index of accessibility to radial highways
- TAX: full-value property-tax rate per $10,000
- PTRATIO: pupil-teacher ratio by town
- B: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
- LSTAT: % lower status of the population

## Predict the house price given the following characteristics (before manipulation!!)

Make sure to transform your variables as needed!

- CRIM: 0.15
- INDUS: 6.07
- CHAS: 1        
- RM:  6.1
- AGE: 33.2
- DIS: 7.6
- PTRATIO: 17
- B: 383
- LSTAT: 10.87
- RAD: 8
- TAX: 284

In [104]:
lcrim = np.log(.15)
ldis = np.log(7.6)
lindus = np.log(6.07)
llstat = np.log(10.87)
lptratio = np.log(17)

# minmax scaling
B = (383-min(b))/(max(b)-min(b))
CRIM = (lcrim-min(logcrim))/(max(logcrim)-min(logcrim))
DIS = (ldis-min(logdis))/(max(logdis)-min(logdis))

#standardization
AGE = (33.2-np.mean(age))/np.sqrt(np.var(age))
INDUS = (lindus-np.mean(logindus))/np.sqrt(np.var(logindus))
LSTAT = (llstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))
PTRATIO = (lptratio-np.mean(logptratio))/(np.sqrt(np.var(logptratio)))

preds = np.array([[CRIM, INDUS, 1, 6.1, AGE, DIS, PTRATIO, B, LSTAT, 0, 0, 1]])

linereg.predict(preds)


array([1501.6826008])

## Summary
Congratulations! You've fitted your first multiple linear regression model on the Boston Housing Data.